In [2]:
import mne
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import pandas as pd

# Training
unfiltered_subjects = ['Aanand_19_01.csv','subhojit_24_02.csv','Abhijeet_6205876348.csv','Nishkarsh_22_01.csv','Aanand_19_02.csv',
                       'Deepanshu_21_1.csv',  'Deepanshu_21_02.csv', 'Mantavya_20_01.csv', 'subhojit_24_01.csv',  'Adit_18_02.csv',
                       'Mantavya_20_02.csv','Dipanjan_21_01.csv', 'Pritesh_22_01.csv', 'Rishab_20_02.csv', 'mohitt.csv',
                       'Rishab_20_01.csv', 'Ritwik_23_02.csv', 'Dipanjan_21_02.csv', 'Nishkarsh_22_02.csv','Pritesh_22_02.csv']
sfreq = 120
ch_names = ['timestamps','Fp1','Fp2','C3','C4','P7','P8','O1','O2','F7','F8','F3','F4',
            'T7','T8','P3','P4','stim','sfreq']

all_data = pd.DataFrame(columns=ch_names)

for file_name in unfiltered_subjects:
    df = pd.read_csv(file_name)  # Specify index column
    df['timestamps'] = df['timestamps'] - df['timestamps'][0]
    
    lvl1_sub = df[(df['timestamps'] >= 0) & (df['timestamps'] < 150)].copy()
    lvl2_sub = df[(df['timestamps'] >= 150) & (df['timestamps'] < 280)].copy()
    lvl3_sub = df[(df['timestamps'] >= 280) & (df['timestamps'] < 400)].copy()

    lvl1_sub['label'] = 1
    lvl2_sub['label'] = 2
    lvl3_sub['label'] = 3
    
    combined_df = pd.concat([lvl1_sub, lvl2_sub, lvl3_sub])
    all_data = pd.concat([all_data, combined_df])


testing_subjects = ['Achal_19_01.csv','Ritwik_23_01.csv']
testing_data = pd.DataFrame(columns=ch_names)

for file_name in testing_subjects:
    df = pd.read_csv(file_name)  # Specify index column
    df['timestamps'] = df['timestamps'] - df['timestamps'][0]
    
    lvl1_sub = df[(df['timestamps'] >= 10) & (df['timestamps'] < 90)].copy()
    lvl2_sub = df[(df['timestamps'] >= 180) & (df['timestamps'] < 260)].copy()
    lvl3_sub = df[(df['timestamps'] >= 300) & (df['timestamps'] < 400)].copy()

    lvl1_sub['label'] = 1
    lvl2_sub['label'] = 2
    lvl3_sub['label'] = 3
    
    combined_df = pd.concat([lvl1_sub, lvl2_sub, lvl3_sub])
    testing_data = pd.concat([testing_data, combined_df])


C:\Users\LEGION\AppData\Local\Temp\ipykernel_22084\3236738338.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_data = pd.concat([all_data, combined_df])
C:\Users\LEGION\AppData\Local\Temp\ipykernel_22084\3236738338.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  testing_data = pd.concat([testing_data, combined_df])


In [4]:
drop1 = all_data.drop(['sfreq'], axis=1)
df1 = drop1.drop(['stim'], axis=1)
df = df1.drop(['label'], axis=1)

drop_data = testing_data.drop(['sfreq'], axis=1)
df2 = drop_data.drop(['stim'], axis=1)
testing = df2.drop(['label'], axis=1)

In [5]:
eeg_data = df.iloc[:,1:18].values.T
eeg_data[:-1] = eeg_data[:-1] * (10*(-6))
eeg_data[:-1] = eeg_data[:-1] / 4

ch_names = ['Fp1',	'Fp2'	,'C3'	,'C4'	,'P7'	,'P8'	,'O1'	,'O2'	,'F7'	,'F8'	,'F3'	,'F4'	,'T7'	,'T8'	,'P3'	,'P4']
info = mne.create_info(ch_names = ch_names, sfreq = sfreq, ch_types = ( ["eeg" ]*16))
raw = mne.io.RawArray(eeg_data, info)
raw.set_eeg_reference()

raw.filter(l_freq = 1,h_freq = 30)

Creating RawArray with float64 data, n_channels=16, n_times=985812
    Range : 0 ... 985811 =      0.000 ...  8215.092 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 397 samples (3.308 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.3s finished


Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,Not available
Good channels,16 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,120.00 Hz
Highpass,1.00 Hz
Lowpass,30.00 Hz


In [6]:
import mne

# EEG data preprocessing for testing
testing_eeg_data = testing.iloc[:, 1:18].values.T  # Selecting EEG channels only
testing_eeg_data[:-1] = testing_eeg_data[:-1] * (10 ** (-6))  # Convert to microvolts
testing_eeg_data[:-1] = testing_eeg_data[:-1] / 4  # Divide by 4

# Create channel names
ch_names = ['Fp1', 'Fp2', 'C3', 'C4', 'P7', 'P8', 'O1', 'O2', 'F7', 'F8', 'F3', 'F4', 'T7', 'T8', 'P3', 'P4']

info_testing = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=["eeg"] * 16)
raw_testing = mne.io.RawArray(testing_eeg_data, info_testing)
raw_testing.set_eeg_reference()
raw_testing.filter(l_freq=1, h_freq=30)


Creating RawArray with float64 data, n_channels=16, n_times=58146
    Range : 0 ... 58145 =      0.000 ...   484.542 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 397 samples (3.308 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.0s finished


Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,Not available
Good channels,16 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,120.00 Hz
Highpass,1.00 Hz
Lowpass,30.00 Hz


In [7]:
duration = 0.6
epochs=mne.make_fixed_length_epochs(raw, duration = duration, overlap=0,preload=True)

Not setting metadata
13691 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 13691 events and 72 original time points ...
0 bad epochs dropped


In [8]:
duration = 0.6

testing_epochs = mne.make_fixed_length_epochs(raw_testing, duration=duration, overlap=0, preload=True)


Not setting metadata
807 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 807 events and 72 original time points ...
0 bad epochs dropped


In [9]:
epochs.drop_bad()  # This will drop bad epochs
num_epochs = len(epochs)
print("Total number of epochs created:", num_epochs)

Total number of epochs created: 13691


In [10]:
testing_epochs.drop_bad()  # This will drop bad epochs
testing_num_epochs = len(testing_epochs)
print("Total number of epochs created for testing:", testing_num_epochs)

Total number of epochs created for testing: 807


In [11]:
import mne

eeg_data = epochs.get_data()

condition_ranges = {'lev1': (0, (num_epochs/3)),
                    'lev2': ((num_epochs/3) , 2*((num_epochs)/3) ),
                    'lev3': ( 2*((num_epochs)/3) , (num_epochs) )}

epochs_by_condition = {}
selected_epochs = []

for condition, (start_epoch, end_epoch) in condition_ranges.items():
    start_epoch = int(start_epoch)
    end_epoch = int(end_epoch)
    condition_epochs = epochs[start_epoch:end_epoch+1].copy()
    epochs_by_condition[condition] = condition_epochs
    selected_epochs.append(condition_epochs)

selected_epochs = mne.concatenate_epochs(selected_epochs)

for condition, epoch_data in epochs_by_condition.items():
    num_epochs = len(epoch_data)
    print(f"Number of epochs for {condition}: {num_epochs}")

print(f"Total number of selected epochs: {len(selected_epochs)}")


Not setting metadata
13693 matching events found
No baseline correction applied


C:\Users\LEGION\AppData\Local\Temp\ipykernel_22084\3628746636.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  eeg_data = epochs.get_data()


Number of epochs for lev1: 4564
Number of epochs for lev2: 4565
Number of epochs for lev3: 4564
Total number of selected epochs: 13693


In [12]:

condition_of_interest = 'lev1'
epoch_array_for_condition = epochs_by_condition[condition_of_interest]
print(f"Epoch array shape for condition {condition_of_interest}: {epoch_array_for_condition.get_data().shape}")


Epoch array shape for condition lev1: (4564, 16, 72)


C:\Users\LEGION\AppData\Local\Temp\ipykernel_22084\1489624503.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  print(f"Epoch array shape for condition {condition_of_interest}: {epoch_array_for_condition.get_data().shape}")


In [13]:
# Define condition ranges based on the number of epochs in the testing data
num_epochs = len(testing_epochs)
condition_ranges_testing = {
    'lev1': (0, num_epochs // 3),
    'lev2': (num_epochs // 3, 2 * (num_epochs // 3)),
    'lev3': (2 * (num_epochs // 3), num_epochs)
}

epochs_by_condition_testing = {}
selected_epochs_testing = []

# Iterate over condition ranges and select epochs for each condition
for condition, (start_epoch, end_epoch) in condition_ranges_testing.items():
    start_epoch = int(start_epoch)
    end_epoch = int(end_epoch)
    condition_epochs = testing_epochs[start_epoch:end_epoch].copy()
    epochs_by_condition_testing[condition] = condition_epochs
    selected_epochs_testing.append(condition_epochs)

# Concatenate epochs from different conditions into a single Epochs object
selected_epochs_testing = mne.concatenate_epochs(selected_epochs_testing)

# Print the number of epochs for each condition
for condition, epoch_data in epochs_by_condition_testing.items():
    num_epochs_condition = len(epoch_data)
    print(f"Number of epochs for {condition}: {num_epochs_condition}")

# Print the total number of selected epochs
print(f"Total number of selected epochs for testing: {len(selected_epochs_testing)}")


Not setting metadata
807 matching events found
No baseline correction applied
Number of epochs for lev1: 269
Number of epochs for lev2: 269
Number of epochs for lev3: 269
Total number of selected epochs for testing: 807


In [30]:
import pandas as pd
import numpy as np
from scipy.signal import welch



def calculate_psd_features(epoch_data, sfreq):
    psd_frequencies, psd_values = welch(epoch_data, fs=sfreq)
    psd_means = np.mean(psd_values, axis=1)  # Calculate mean PSD values for each channel
    psd_variances = np.var(psd_values, axis=1)  # Calculate variance of PSD values for each channel
    psd_std_devs = np.std(psd_values, axis=1)  # Calculate standard deviation of PSD values for each channel

    freq_min = 14
    freq_max = 30

    idx_min = np.argmax(psd_frequencies > freq_min) - 1
    idx_max = np.argmax(psd_frequencies > freq_max)

    psd_values_alpha_band = psd_values[:, idx_min:idx_max]
    psd_means_alpha_band = np.mean(psd_values_alpha_band, axis=1)  # Calculate mean PSD values in alpha band for each channel
    psd_variances_alpha_band = np.var(psd_values_alpha_band, axis=1)  # Calculate variance of PSD values in alpha band for each channel
    psd_std_devs_alpha_band = np.std(psd_values_alpha_band, axis=1)  # Calculate standard deviation of PSD values in alpha band for each channel
    return psd_means_alpha_band, psd_variances_alpha_band, psd_std_devs_alpha_band

channel_names = epochs.ch_names
sfreq = epochs.info['sfreq']  # Sampling frequency

columns = [f'{feature}_{channel}' for feature in ['mean_psd', 'variance_psd', 'standarddev_psd'] for channel in channel_names]
columns.append('condition')
feature_df = pd.DataFrame(columns=columns)

def append_epoch_data(epoch_array, condition_label):
    global feature_df
    for epoch in epoch_array:
        psd_means, psd_variances, psd_std_devs = calculate_psd_features(epoch, sfreq)
        epoch_features = np.concatenate([psd_means, psd_variances, psd_std_devs])
        epoch_features = np.append(epoch_features, condition_label)
        epoch_df = pd.DataFrame([epoch_features], columns=columns)
        feature_df = pd.concat([feature_df, epoch_df], ignore_index=True)

append_epoch_data(epochs_by_condition['lev1'], 'lev1')
append_epoch_data(epochs_by_condition['lev2'], 'lev2')
append_epoch_data(epochs_by_condition['lev3'], 'lev3')

print("Shape of feature DataFrame:", feature_df.shape)
print(feature_df.head())


c:\Users\LEGION\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\signal\_spectral_py.py:1999: UserWarning: nperseg = 256 is greater than input length  = 12, using nperseg = 12
  warnings.warn('nperseg = {0:d} is greater than input length '


Shape of feature DataFrame: (41077, 49)
         mean_psd_Fp1        mean_psd_Fp2         mean_psd_C3  \
0  1246184.5037780665   1037955.895575066   7175.706249993606   
1  303669.78597822407  257913.25218291095  2479.4195899185816   
2  241147.12225637794  205893.62456607868   2916.149422271106   
3   163541.2222952666  135454.03020143762   2547.627773784771   
4  119394.55188332939  103581.14034804993   2676.135294472259   

          mean_psd_C4         mean_psd_P7         mean_psd_P8  \
0   527107.5501359972   979465.7994063155   72133.93933336825   
1  126980.62292119682  242134.10468745147  21616.710304816374   
2   104131.3618447669  192269.43556531277   16865.37658551462   
3   68055.23370362402  124299.43834807091   10419.90351987869   
4   56327.55941638214   95194.61512789618  10386.012392758015   

          mean_psd_O1         mean_psd_O2         mean_psd_F7  \
0  482802.96348860365  33277.607199607985  443423.00056077837   
1   69123.27742951916     6669.3400224457   1019

In [31]:
# Rename columns using a dictionary
new_column_names = {
    'mean_psd_Fp1': 'mean_psd_Fp1',
    'mean_psd_Fp2': 'mean_psd_Fp2',
    'mean_psd_C3': 'mean_psd_C3',
    'mean_psd_C4': 'mean_psd_C4',
    'mean_psd_P7': 'mean_psd_P7',
    'mean_psd_P8': 'mean_psd_P8',
    'mean_psd_O1': 'mean_psd_O1',
    'mean_psd_O2': 'mean_psd_O2',
    'mean_psd_F7': 'mean_psd_F7',
    'mean_psd_F8': 'mean_psd_F8',
    'standarddev_psd_Fp1': 'stddev_psd_Fp1',
    'standarddev_psd_Fp2': 'stddev_psd_Fp2',
    'standarddev_psd_C3': 'stddev_psd_C3',
    'standarddev_psd_C4': 'stddev_psd_C4',
    'standarddev_psd_P7': 'stddev_psd_P7',
    'standarddev_psd_P8': 'stddev_psd_P8',
    'standarddev_psd_O1': 'stddev_psd_O1',
    'standarddev_psd_O2': 'stddev_psd_O2',
    'standarddev_psd_F7': 'stddev_psd_F7',
    'standarddev_psd_F8': 'stddev_psd_F8'
}

# Rename columns using the rename() method
feature_df.rename(columns=new_column_names, inplace=True)

# Print the DataFrame
print(feature_df.head())


         mean_psd_Fp1        mean_psd_Fp2         mean_psd_C3  \
0  1246184.5037780665   1037955.895575066   7175.706249993606   
1  303669.78597822407  257913.25218291095  2479.4195899185816   
2  241147.12225637794  205893.62456607868   2916.149422271106   
3   163541.2222952666  135454.03020143762   2547.627773784771   
4  119394.55188332939  103581.14034804993   2676.135294472259   

          mean_psd_C4         mean_psd_P7         mean_psd_P8  \
0   527107.5501359972   979465.7994063155   72133.93933336825   
1  126980.62292119682  242134.10468745147  21616.710304816374   
2   104131.3618447669  192269.43556531277   16865.37658551462   
3   68055.23370362402  124299.43834807091   10419.90351987869   
4   56327.55941638214   95194.61512789618  10386.012392758015   

          mean_psd_O1         mean_psd_O2         mean_psd_F7  \
0  482802.96348860365  33277.607199607985  443423.00056077837   
1   69123.27742951916     6669.3400224457   101933.1824656781   
2   73358.20257738318  

In [32]:
feature_df

,mean_psd_Fp1,mean_psd_Fp2,mean_psd_C3,mean_psd_C4,mean_psd_P7,mean_psd_P8,mean_psd_O1,mean_psd_O2,mean_psd_F7,mean_psd_F8,...,stddev_psd_O2,stddev_psd_F7,stddev_psd_F8,standarddev_psd_F3,standarddev_psd_F4,standarddev_psd_T7,standarddev_psd_T8,standarddev_psd_P3,standarddev_psd_P4,condition
0,1246184.5037780665,1037955.895575066,7175.706249993606,527107.5501359972,979465.7994063155,72133.93933336825,482802.96348860365,33277.607199607985,443423.00056077837,79844.36284495927,...,35698.92130270356,478011.31427987624,65696.37058149357,57786.962721179734,3487855.0870026886,88968.58070669783,182696296.9452925,94016.63551036333,55712811.51443904,lev1
1,303669.78597822407,257913.25218291095,2479.4195899185816,126980.62292119682,242134.10468745147,21616.710304816374,69123.27742951916,6669.3400224457,101933.1824656781,14025.551046066297,...,3521.441019313134,133407.53721432202,7093.609846477616,4007.4620483235694,948543.7728921876,11290.518950756998,49409912.01441849,15736.666538218915,15368162.585603695,lev1
2,241147.12225637794,205893.62456607868,2916.149422271106,104131.3618447669,192269.43556531277,16865.37658551462,73358.20257738318,6237.807460374867,74224.83022905745,7521.736936673413,...,4737.927034794957,93125.20847259018,2574.4485668592856,2001.6503812835836,844427.0203422116,14966.643905355288,39976558.882050864,17670.27840290237,12911219.264737349,lev1
3,163541.2222952666,135454.03020143762,2547.627773784771,68055.23370362402,124299.43834807091,10419.90351987869,79231.47980395457,7013.308972464602,60055.33451005866,12259.535218412331,...,3704.6798514468255,79754.23740724556,5997.873612199568,3978.5472161307807,576723.0507806854,3899.2407637627202,30093968.941779487,5787.157765578282,9287000.265037993,lev1
4,119394.55188332939,103581.14034804993,2676.135294472259,56327.55941638214,95194.61512789618,10386.012392758015,55903.358216350985,4393.5976330710755,41881.40978672828,7661.934306000854,...,748.150788548027,50762.09017511015,3061.6911817423625,4946.392934497634,388573.13796467165,4766.942695446411,19313075.999070544,5671.823833215368,6116845.302462846,lev1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41072,2091.7254702995647,2003.3800132225724,1964.6615154437495,2065.293254368537,2141.877672425315,2614.5518705957443,1370.382815875855,2545.794029401874,2743.0210381971774,2651.9401390449207,...,1754.195464349822,2034.6890084114625,1915.4106206122262,2036.915954910594,2030.7123164633829,2177.9195429162187,1741.0535569370727,1928.947544150983,2784.668045872555,lev3
41073,6142.798895328199,6097.554349152015,5844.74497733734,6101.392427346057,6024.6980597847905,10194.597862896047,3103.8462808583986,6122.324380376037,12997.551847576304,13352.384893758697,...,3681.4419054460827,6868.873371717962,7053.853133835797,6916.738507253842,7024.617727234137,1465.9916280842133,6657.074531410777,4693.996122574334,15293.071105345136,lev3
41074,3582.770385550683,3615.9351332911683,3600.8372876466815,3565.226697074888,3345.246654361478,4057.8978071583933,1744.3717490872014,3797.809307959678,6156.393361888481,6274.913214153005,...,2454.3350709099673,4499.778747759991,4568.247165250638,4542.964521247883,4616.953764323264,1657.251370391382,4639.055713386789,1688.574969676199,9289.219905680324,lev3
41075,7115.54756936757,7029.896029837073,7142.357507745274,7256.858812442842,7353.343719316316,8862.050491863798,3699.137345860269,6409.092272504819,9173.953927977325,9187.94111123388,...,1916.2451291252871,2674.301421579577,2873.4721089364084,2772.2301328825533,2908.700044898885,4894.242259496755,2323.3934956154408,972.490296466485,10781.9822550387,lev3


In [33]:
feature_columns = [f'{feature}_{channel}' for feature in ['mean_psd', 'variance_psd', 'standarddev_psd'] for channel in ch_names]
feature_columns.append('condition') 
testing_feature_df = pd.DataFrame(columns=feature_columns)

def append_testing_epoch_data(epoch_array, condition_label):
    global testing_feature_df
    for epoch in epoch_array:
        psd_means, psd_variances, psd_std_devs = calculate_psd_features(epoch, sfreq)
        epoch_features = np.concatenate([psd_means, psd_variances, psd_std_devs])
        epoch_features = np.append(epoch_features, condition_label)
        epoch_df = pd.DataFrame([epoch_features], columns=feature_columns)
        testing_feature_df = pd.concat([testing_feature_df, epoch_df], ignore_index=True)

append_testing_epoch_data(epochs_by_condition_testing['lev1'], 'lev1')
append_testing_epoch_data(epochs_by_condition_testing['lev2'], 'lev2')
append_testing_epoch_data(epochs_by_condition_testing['lev3'], 'lev3')

print("Shape of testing feature DataFrame:", testing_feature_df.shape)
print(testing_feature_df.head())


c:\Users\LEGION\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\signal\_spectral_py.py:1999: UserWarning: nperseg = 256 is greater than input length  = 72, using nperseg = 72
  warnings.warn('nperseg = {0:d} is greater than input length '


Shape of testing feature DataFrame: (807, 49)
           mean_psd_Fp1          mean_psd_Fp2           mean_psd_C3  \
0   0.04239833015625566   0.04239834639545974  0.042398340336458645   
1  0.016822087396385115  0.016822094269049678  0.016822086821315033   
2  0.009600383680736619  0.009600394036725766  0.009600386169948998   
3  0.016351245459510274  0.016351257231996055  0.016351245036358417   
4   0.00615507403547462  0.006155072909108814  0.006155070001453949   

             mean_psd_C4            mean_psd_P7           mean_psd_P8  \
0    0.04239833914104005   0.042398392746926666  0.042398345840275124   
1    0.01682207229513797    0.01682208623358466  0.016822098940331976   
2   0.009600384216052357   0.009600381180564678  0.009600381122960282   
3    0.01635123752298921   0.016351301642956152  0.016351249367365293   
4  0.0061550705663853825  0.0061550999601178305  0.006155075799354056   

            mean_psd_O1           mean_psd_O2           mean_psd_F7  \
0   0.04239833593

In [34]:
new_column_names = {
    'mean_psd_Fp1': 'mean_psd_Fp1',
    'mean_psd_Fp2': 'mean_psd_Fp2',
    'mean_psd_C3': 'mean_psd_C3',
    'mean_psd_C4': 'mean_psd_C4',
    'mean_psd_P7': 'mean_psd_P7',
    'mean_psd_P8': 'mean_psd_P8',
    'mean_psd_O1': 'mean_psd_O1',
    'mean_psd_O2': 'mean_psd_O2',
    'mean_psd_F7': 'mean_psd_F7',
    'mean_psd_F8': 'mean_psd_F8',
    'standarddev_psd_Fp1': 'stddev_psd_Fp1',
    'standarddev_psd_Fp2': 'stddev_psd_Fp2',
    'standarddev_psd_C3': 'stddev_psd_C3',
    'standarddev_psd_C4': 'stddev_psd_C4',
    'standarddev_psd_P7': 'stddev_psd_P7',
    'standarddev_psd_P8': 'stddev_psd_P8',
    'standarddev_psd_O1': 'stddev_psd_O1',
    'standarddev_psd_O2': 'stddev_psd_O2',
    'standarddev_psd_F7': 'stddev_psd_F7',
    'standarddev_psd_F8': 'stddev_psd_F8'
}

testing_feature_df.rename(columns=new_column_names, inplace=True)

print(testing_feature_df.head())


           mean_psd_Fp1          mean_psd_Fp2           mean_psd_C3  \
0   0.04239833015625566   0.04239834639545974  0.042398340336458645   
1  0.016822087396385115  0.016822094269049678  0.016822086821315033   
2  0.009600383680736619  0.009600394036725766  0.009600386169948998   
3  0.016351245459510274  0.016351257231996055  0.016351245036358417   
4   0.00615507403547462  0.006155072909108814  0.006155070001453949   

             mean_psd_C4            mean_psd_P7           mean_psd_P8  \
0    0.04239833914104005   0.042398392746926666  0.042398345840275124   
1    0.01682207229513797    0.01682208623358466  0.016822098940331976   
2   0.009600384216052357   0.009600381180564678  0.009600381122960282   
3    0.01635123752298921   0.016351301642956152  0.016351249367365293   
4  0.0061550705663853825  0.0061550999601178305  0.006155075799354056   

            mean_psd_O1           mean_psd_O2           mean_psd_F7  \
0   0.04239833593873676  0.042398279984934055   0.0424056651363

In [35]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

X = feature_df.drop(columns=['condition'])
y = feature_df['condition']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

gb_classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)

gb_classifier.fit(X_train, y_train)

y_pred = gb_classifier.predict(X_test)

#Getting inter subject feature
X_testing = testing_feature_df.drop(columns=['condition'])
y_testing = testing_feature_df['condition']

y_pred_testing = gb_classifier.predict(X_testing)

accuracy_testing = accuracy_score(y_testing, y_pred_testing)
print("Accuracy on testing data:", accuracy_testing)

Accuracy on testing data: 0.19454770755885997


In [36]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

X = feature_df.drop(columns=['condition'])
y = feature_df['condition']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

svm_classifier = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)

svm_classifier.fit(X_train, y_train)

y_pred = svm_classifier.predict(X_test)

#Getting inter subject features

X_testing = testing_feature_df.drop(columns=['condition'])
y_testing = testing_feature_df['condition']

y_pred_testing = svm_classifier.predict(X_testing)

accuracy_testing = accuracy_score(y_testing, y_pred_testing)
print("Accuracy on testing data:", accuracy_testing)


Accuracy on testing data: 0.3333333333333333


In [21]:
# def append_epoch_data(epoch_array, condition_label):
#         global feature_df
#         for epoch in epoch_array:
#             psd_means, psd_variances, psd_std_devs = calculate_psd_features(epoch, sfreq)
#             epoch_features = np.concatenate([psd_means, psd_variances, psd_std_devs])
#             epoch_features = np.append(epoch_features, condition_label)
#             epoch_df = pd.DataFrame([epoch_features], columns=columns)
#             feature_df = pd.concat([feature_df, epoch_df], ignore_index=True)

# def calculate_psd_features(epoch_data, sfreq):
#         psd_frequencies, psd_values = welch(epoch_data, fs=sfreq)
#         psd_means = np.mean(psd_values, axis=1)  # Calculate mean PSD values for each channel
#         psd_variances = np.var(psd_values, axis=1)  # Calculate variance of PSD values for each channel
#         psd_std_devs = np.std(psd_values, axis=1)  # Calculate standard deviation of PSD values for each channel

#         freq_min = 14
#         freq_max = 30

#         # Find the indices corresponding to the frequency range of interest
#         idx_min = np.argmax(psd_frequencies > freq_min) - 1
#         idx_max = np.argmax(psd_frequencies > freq_max)

#         # Extract the PSD within the specified frequency range
#         psd_values_alpha_band = psd_values[:, idx_min:idx_max]
#         psd_means_alpha_band = np.mean(psd_values_alpha_band, axis=1)  # Calculate mean PSD values in alpha band for each channel
#         psd_variances_alpha_band = np.var(psd_values_alpha_band, axis=1)  # Calculate variance of PSD values in alpha band for each channel
#         psd_std_devs_alpha_band = np.std(psd_values_alpha_band, axis=1)  # Calculate standard deviation of PSD values in alpha band for each channel
#         return psd_means_alpha_band, psd_variances_alpha_band, psd_std_devs_alpha_band


In [22]:
# import matplotlib.pyplot as plt


# durations = []
# accuracies = []

# duration_range = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

# for duration in duration_range:
#     epochs = mne.make_fixed_length_epochs(raw, duration=duration, overlap=0, preload=True)
    
#     num_epochs = len(epochs)

#     eeg_data = epochs.get_data()

#     # condition_ranges = {'lev1': (0, (num_epochs/3)),
#     #                     'lev2': ((num_epochs/3) , ((2*num_epochs)/3) ),
#     #                     'lev3': ((2*num_epochs)/3 , (num_epochs) )}
#     condition_ranges = {'lev1': (0, (num_epochs/6)),
#                     'lev2': (3*(num_epochs/6) , 4*((num_epochs)/6) ),
#                     'lev3': ( 5*((num_epochs)/6) , (num_epochs) )}

#     epochs_by_condition = {}
#     selected_epochs = []

#     for condition, (start_epoch, end_epoch) in condition_ranges.items():
#         start_epoch = int(start_epoch)
#         end_epoch = int(end_epoch)
#         condition_epochs = epochs[start_epoch:end_epoch+1].copy()
#         epochs_by_condition[condition] = condition_epochs
#         selected_epochs.append(condition_epochs)

#     selected_epochs = mne.concatenate_epochs(selected_epochs)
#     import pandas as pd
#     import numpy as np
#     from scipy.signal import welch
    
#     channel_names = epochs.ch_names
#     sfreq = epochs.info['sfreq']

#     columns = [f'{feature}_{channel}' for feature in ['mean_psd', 'variance_psd', 'standarddev_psd'] for channel in channel_names]
#     columns.append('condition')
#     feature_df = pd.DataFrame(columns=columns)

    

#     append_epoch_data(epochs_by_condition['lev1'], 'lev1')
#     append_epoch_data(epochs_by_condition['lev2'], 'lev2')
#     append_epoch_data(epochs_by_condition['lev3'], 'lev3')
#     # Rename columns using a dictionary
#     new_column_names = {
#         'mean_psd_Fp1': 'mean_psd_Fp1',
#         'mean_psd_Fp2': 'mean_psd_Fp2',
#         'mean_psd_C3': 'mean_psd_C3',
#         'mean_psd_C4': 'mean_psd_C4',
#         'mean_psd_P7': 'mean_psd_P7',
#         'mean_psd_P8': 'mean_psd_P8',
#         'mean_psd_O1': 'mean_psd_O1',
#         'mean_psd_O2': 'mean_psd_O2',
#         'mean_psd_F7': 'mean_psd_F7',
#         'mean_psd_F8': 'mean_psd_F8',
#         'standarddev_psd_Fp1': 'stddev_psd_Fp1',
#         'standarddev_psd_Fp2': 'stddev_psd_Fp2',
#         'standarddev_psd_C3': 'stddev_psd_C3',
#         'standarddev_psd_C4': 'stddev_psd_C4',
#         'standarddev_psd_P7': 'stddev_psd_P7',
#         'standarddev_psd_P8': 'stddev_psd_P8',
#         'standarddev_psd_O1': 'stddev_psd_O1',
#         'standarddev_psd_O2': 'stddev_psd_O2',
#         'standarddev_psd_F7': 'stddev_psd_F7',
#         'standarddev_psd_F8': 'stddev_psd_F8'
#     }

#     feature_df.rename(columns=new_column_names, inplace=True)

#     X = feature_df.drop(columns=['condition'])
#     y = feature_df['condition']
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#     rf_classifier.fit(X_train, y_train)
#     y_pred = rf_classifier.predict(X_test)
#     accuracy = accuracy_score(y_test, y_pred)
    
#     durations.append(duration)
#     accuracies.append(accuracy)

# plt.plot(durations, accuracies, marker='o')
# plt.xlabel('Duration')
# plt.ylabel('Accuracy')
# plt.title('Duration vs Accuracy')
# plt.grid(True)
# plt.show()


Not setting metadata
82151 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 82151 events and 12 original time points ...
0 bad epochs dropped


C:\Users\LEGION\AppData\Local\Temp\ipykernel_22084\4096582750.py:14: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  eeg_data = epochs.get_data()


Not setting metadata
41077 matching events found
No baseline correction applied


c:\Users\LEGION\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\signal\_spectral_py.py:1999: UserWarning: nperseg = 256 is greater than input length  = 12, using nperseg = 12
  warnings.warn('nperseg = {0:d} is greater than input length '


KeyboardInterrupt: 

In [ ]:
# import matplotlib.pyplot as plt

# # Initialize lists to store durations and accuracies
# durations = []
# accuracies = []

# # Define a range of durations to try
# duration_range = [0.05,0.1, 0.2, 0.3, 0.4, 0.5,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3,1.4,1.5,2,3]

# for duration in duration_range:
#     epochs = mne.make_fixed_length_epochs(raw, duration=duration, overlap=0, preload=True)
    
#     num_epochs = len(epochs)

#     eeg_data = epochs.get_data()

#     # condition_ranges = {'lev1': (0, (num_epochs/3)),
#     #                     'lev2': ((num_epochs/3) , ((2*num_epochs)/3) ),
#     #                     'lev3': ((2*num_epochs)/3 , (num_epochs) )}
#     condition_ranges = {'lev1': (0, (num_epochs/6)),
#                     'lev2': (3*(num_epochs/6) , 4*((num_epochs)/6) ),
#                     'lev3': ( 5*((num_epochs)/6) , (num_epochs) )}

#     epochs_by_condition = {}
#     selected_epochs = []

#     for condition, (start_epoch, end_epoch) in condition_ranges.items():
#         start_epoch = int(start_epoch)
#         end_epoch = int(end_epoch)
#         condition_epochs = epochs[start_epoch:end_epoch+1].copy()
#         epochs_by_condition[condition] = condition_epochs
#         selected_epochs.append(condition_epochs)

#     selected_epochs = mne.concatenate_epochs(selected_epochs)
#     import pandas as pd
#     import numpy as np
#     from scipy.signal import welch



    
#     channel_names = epochs.ch_names
#     sfreq = epochs.info['sfreq']  # Sampling frequency

#     columns = [f'{feature}_{channel}' for feature in ['mean_psd', 'variance_psd', 'standarddev_psd'] for channel in channel_names]
#     columns.append('condition')
#     feature_df = pd.DataFrame(columns=columns)

    

#     append_epoch_data(epochs_by_condition['lev1'], 'lev1')
#     append_epoch_data(epochs_by_condition['lev2'], 'lev2')
#     append_epoch_data(epochs_by_condition['lev3'], 'lev3')
#     # Rename columns using a dictionary
#     new_column_names = {
#         'mean_psd_Fp1': 'mean_psd_Fp1',
#         'mean_psd_Fp2': 'mean_psd_Fp2',
#         'mean_psd_C3': 'mean_psd_C3',
#         'mean_psd_C4': 'mean_psd_C4',
#         'mean_psd_P7': 'mean_psd_P7',
#         'mean_psd_P8': 'mean_psd_P8',
#         'mean_psd_O1': 'mean_psd_O1',
#         'mean_psd_O2': 'mean_psd_O2',
#         'mean_psd_F7': 'mean_psd_F7',
#         'mean_psd_F8': 'mean_psd_F8',
#         'standarddev_psd_Fp1': 'stddev_psd_Fp1',
#         'standarddev_psd_Fp2': 'stddev_psd_Fp2',
#         'standarddev_psd_C3': 'stddev_psd_C3',
#         'standarddev_psd_C4': 'stddev_psd_C4',
#         'standarddev_psd_P7': 'stddev_psd_P7',
#         'standarddev_psd_P8': 'stddev_psd_P8',
#         'standarddev_psd_O1': 'stddev_psd_O1',
#         'standarddev_psd_O2': 'stddev_psd_O2',
#         'standarddev_psd_F7': 'stddev_psd_F7',
#         'standarddev_psd_F8': 'stddev_psd_F8'
#     }

#     # Rename columns using the rename() method
#     feature_df.rename(columns=new_column_names, inplace=True)
#     # Ensure the rest of your code is within this loop so that it executes for each duration
    
#     # Train the classifier and calculate accuracy
#     X = feature_df.drop(columns=['condition'])
#     y = feature_df['condition']
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    

#     # Initialize the Gradient Boosting classifier
#     gb_classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)

#     # Train the classifier on the training data
#     gb_classifier.fit(X_train, y_train)

#     # Predict the labels for the test set
#     y_pred = gb_classifier.predict(X_test)

#     # Calculate the accuracy of the model
#     accuracy = accuracy_score(y_test, y_pred)
    
#     # Append duration and accuracy to lists
#     durations.append(duration)
#     accuracies.append(accuracy)

# # Plot duration vs accuracy
# plt.plot(durations, accuracies, marker='o')
# plt.xlabel('Duration')
# plt.ylabel('Accuracy')
# plt.title('Duration vs Accuracy')
# plt.grid(True)
# plt.show()
